# Youtube url -> Transcription -> Summary
- using whisper-small, and "knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM"

### Setting up whisper and BART:

In [1]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"

#whisper
model_transcriber = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-small",
  chunk_length_s=30,
  device=device,
)

def transcribe(file, model=model_transcriber, language="english"):
    return model(file, generate_kwargs={"language": language})["text"]


#BART:
model_summarizer = pipeline("summarization", model="knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM", device=device)
def summarize(txt,model=model_summarizer):
  return model(txt)[0]['summary_text']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

### Youtube -> Mp4 -> Mp3 -> Flac pipeline:

In [2]:
!pip install yt-dlp -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.0 MB/s eta 0:00:00


In [3]:
from yt_dlp import YoutubeDL, DownloadError
import os

def dl_flac(url, output_path): #function to download .flac from youtube
    if not os.path.exists(output_path):
      os.makedirs(output_path)
    ydl_opts = {
        'format': 'bestaudio/best', #change params as needed
        "max_filesize": 20 * 1024 * 1024,
         "outtmpl": f"{output_path}/%(id)s.%(ext)s",
        "noplaylist": True,
        "verbose": False,
        "quiet": True,
        "no_warnings": True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'flac',
            'preferredquality': '192',
        }],
    }
    ydl = YoutubeDL(ydl_opts)
    try:
        meta = ydl.extract_info(
            url,
            download=True,
        )
    except DownloadError as e:
        raise e
    else:
        video_id = meta["id"]
        return { #we return a dict
            "title": meta["title"],
            "file_path": f"{output_path}/{video_id}.flac"
        }

### Running inference:

In [5]:
def inference(URL,verbose=False,prnt=False): #in prod, use default settings
  if verbose: print('Downloading from Youtube...')
  flac_path = dl_flac(URL,'/downloaded_flac')['file_path']

  if verbose: print('Transcribing audio...')
  transcript = transcribe(flac_path)
  if verbose: print('Summarizing transcript...')
  summary = summarize(transcript)
  if prnt: print(f'\nSummary: \n{summary}')
  else: return(summary)

In [6]:
inference('https://www.youtube.com/watch?v=EaQgzQDQeHQ', verbose=True, prnt=True)

ERROR: [youtube] EaQgzQDQeHQ: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


DownloadError: ERROR: [youtube] EaQgzQDQeHQ: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U

In [10]:
!pip install yt-dlp -U

In [7]:
dl_flac('https://www.youtube.com/watch?v=EaQgzQDQeHQ','/')

ERROR: [youtube] EaQgzQDQeHQ: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


DownloadError: ERROR: [youtube] EaQgzQDQeHQ: Failed to extract any player response; please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U